In [5]:
from lexicalrichness import LexicalRichness

text = ["""Measure of textual lexical diversity, computed as the mean length of sequential words in
                a text that maintains a minimum threshold TTR score.

                Iterates over words until TTR scores falls below a threshold, then increase factor
                counter by 1 and start over. McCarthy and Jarvis (2010, pg. 385) recommends a factor
                threshold in the range of [0.660, 0.750].
                (McCarthy 2005, McCarthy and Jarvis 2010)""",'Something random']

# instantiate new text object (use the tokenizer=blobber argument to use the textblob tokenizer)



AttributeError: 'list' object has no attribute 'lower'

In [5]:
import numpy as np
def make_dictionary(dataset):
    d_dict = dataset.to_dict()
    d_list = []
    for i in range(len(d_dict['task_id'])):
        d_list.append({k:d_dict[k][i] for k in d_dict})
    return d_list

In [87]:
from lexicalrichness import LexicalRichness
def calculate_mtld(text):
    lex = LexicalRichness(text)
    try:
        result = lex.mtld(threshold=0.72)
    except ZeroDivisionError as e:
        result = None
    return result

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def calculate_spread_cosine(embeddings):
    """
    Calculate the spread using cosine similarity between embeddings.
    """
    print('Started Cosine Sim')
    # Assuming embeddings is a numpy array where each row represents the embedding of a word
    similarity_matrix = cosine_similarity(embeddings)
    print('Finished Cosine Sim')
    # Exclude self-similarity (diagonal) and take the average of the remaining similarities
    spread_cosine = np.mean(similarity_matrix[np.triu_indices(similarity_matrix.shape[0], k=1)])
    
    return spread_cosine
def normalize_score(score, min_range, max_range):
    """
    Normalize a score within a given range.
    """
    # normalized_score = (score - min_range) / (max_range - min_range)
    return score

def get_MTLD_list(samples):
    probs, mtld_list = [], []
    for i in samples:
        if calculate_mtld(i):
            mtld_list.append(calculate_mtld(i))
        else:
            probs.append(i)
    return probs, mtld_list

In [110]:
len(seed)

NameError: name 'seed' is not defined

In [101]:
import json
import pickle
def calculate_diversity_score(dataset,model,dataset_name, question_key='question', code_key='code'):
    print('Starting for new dataset\n\n')
    min_range = 0.7775569693909751
    max_range = 0.9527710969170251
    questions = [i[question_key] for i in dataset]
    code = [i[code_key] for i in dataset]
    print('Calculating MTLD!')
    question_probs, question_mtld = get_MTLD_list(questions)
    print(f'Length of probs = {len(question_probs)}')
    print(f'Length of MTLD list = {len(question_mtld)}')
    print()
    code_probs, code_mtld = get_MTLD_list(code)
    print(f'Length of probs = {len(code_probs)}')
    print(f'Length of MTLD list = {len(code_mtld)}')
    question_mtld = sum(question_mtld)/len(question_mtld)
    code_mtld = sum(code_mtld)/len(code_mtld)
    sample_num = min(len(questions),5000)
    sampled_numbers = random.sample(range(len(questions)), sample_num)
    sampled_questions = [i for idx, i in enumerate(questions) if idx in sampled_numbers]
    sampled_code = [i for idx, i in enumerate(code) if idx in sampled_numbers]
    print('MTLD Done!')
    print('Preparing embeddings!')
    question_embeddings = model.encode(sampled_questions,show_progress_bar=True, batch_size=512, device='cuda')
    # with open(f'question_embeddings_{dataset_name}.pkl', "wb") as fOut:
    #     pickle.dump({'sentences': questions, 'embeddings': question_embeddings}, fOut, protocol=pickle.HIGHEST_PROTOCOL)
    code_embeddings = model.encode(sampled_code,show_progress_bar=True, batch_size=512, device='cuda')
    # with open(f'code_embeddings_{dataset_name}.pkl', "wb") as fOut:
    #     pickle.dump({'sentences': code, 'embeddings': code_embeddings}, fOut, protocol=pickle.HIGHEST_PROTOCOL)
    print('Embedings done!')
    print('Calculating spread!')
    question_spread = normalize_score(calculate_spread_cosine(question_embeddings), min_range, max_range)
    code_spread = normalize_score(calculate_spread_cosine(code_embeddings), min_range, max_range)
    print('Spread calc done!')
    return question_mtld, code_mtld, question_spread, code_spread
    

In [49]:
from sentence_transformers import SentenceTransformer, util
print('Loading Model!')
model = SentenceTransformer('/workspace/CS762_Project/e5-base-v2', device='cuda')
print('Model Loaded!')

Loading Model!
Model Loaded!


In [50]:
from datasets import load_dataset

evol_codealpaca_v1 = load_dataset("theblackcat102/evol-codealpaca-v1")
evol_instruct_code_v1 = load_dataset("nickrosh/Evol-Instruct-Code-80k-v1")
magicoder_instruct = load_dataset("ise-uiuc/Magicoder-OSS-Instruct-75K")
magicoder_evol_instruct = load_dataset("ise-uiuc/Magicoder-Evol-Instruct-110K")
with open('/workspace/CS762_Project/code_alpaca_20k.json', 'r') as fp:
    code_alpaca = json.load(fp)
with open('/workspace/CS762_Project/Data_files/final_combined_data_Dec15_mod_asserts.json', 'r') as fp:
    code_mix = json.load(fp)

In [51]:
# print(evol_codealpaca_v1['train'][0])
# print(evol_instruct_code_v1['train'][0])
# print(magicoder_instruct['train'][0])
# print(magicoder_evol_instruct['train'][0])
# print(code_alpaca[0])

In [60]:
def filter_python(sample, q_key='question', c_key='code'):
    question = sample[q_key].lower()
    code = sample[c_key].lower()
    if 'python' in question or 'python' in code:
        return True
    return False
    
    

In [52]:
def make_dict_from_dataset(dataset):
    return [i for i in dataset]

In [53]:
evol_codealpaca_v1 = make_dict_from_dataset(evol_codealpaca_v1['train'])
evol_instruct_code_v1 = make_dict_from_dataset(evol_instruct_code_v1['train'])
magicoder_instruct = make_dict_from_dataset(magicoder_instruct['train'])
magicoder_evol_instruct = make_dict_from_dataset(magicoder_evol_instruct['train'])

In [69]:
result_only_python = {}
dataset_vars = [evol_codealpaca_v1, evol_instruct_code_v1, magicoder_instruct, magicoder_evol_instruct, code_alpaca, code_mix]
name_mapping = ['evol_codealpaca_v1','evol_instruct_code_v1','magicoder_instruct','magicoder_evol_instruct','code_alpaca','CodeMix-dataset']
keys = [{'question':'instruction', 'code':'output'}, {'question':'instruction', 'code':'output'}, {'question':'problem', 'code':'solution'}, {'question':'instruction', 'code':'response'},{'question':'instruction', 'code':'output'}, {'question':'question', 'code':'code'}]
for idx1,d in enumerate(dataset_vars):
    print(f'Dataset {name_mapping[idx1]}')
    new_ds = []
    for j in d:
        if filter_python(j, q_key=keys[idx1]['question'], c_key=keys[idx1]['code']) or name_mapping[idx1]=='CodeMix-dataset' or name_mapping[idx1]=='magicoder_instruct':
            new_ds.append(j)
    result_only_python[name_mapping[idx1]] = new_ds

Dataset evol_codealpaca_v1
Dataset evol_instruct_code_v1
Dataset magicoder_instruct
Dataset magicoder_evol_instruct
Dataset code_alpaca
Dataset CodeMix-dataset


In [70]:
{k:len(v) for k,v in result_only_python.items()}

{'evol_codealpaca_v1': 57997,
 'evol_instruct_code_v1': 26588,
 'magicoder_instruct': 75197,
 'magicoder_evol_instruct': 57931,
 'code_alpaca': 2672,
 'CodeMix-dataset': 20252}

In [72]:
evol_codealpaca_v1_python = result_only_python['evol_codealpaca_v1']
evol_instruct_code_v1_python = result_only_python['evol_instruct_code_v1']
magicoder_instruct_python = result_only_python['magicoder_instruct']
magicoder_evol_instruct_python = result_only_python['magicoder_evol_instruct']
code_alpaca_python = result_only_python['code_alpaca']

In [54]:
magicoder_evol_instruct[1].keys()

dict_keys(['instruction', 'response'])

In [76]:
import random

random.sample([1,2,3,4],2)

[1, 4]

In [102]:
result_dict = {}
random.seed(42)
# dataset_vars = [evol_codealpaca_v1, evol_instruct_code_v1, magicoder_instruct, magicoder_evol_instruct, code_alpaca, code_mix]
dataset_vars = [evol_codealpaca_v1_python, evol_instruct_code_v1_python, magicoder_instruct_python, magicoder_evol_instruct_python, code_alpaca_python, code_mix]
name_mapping = ['evol_codealpaca_v1','evol_instruct_code_v1','magicoder_instruct','magicoder_evol_instruct','code_alpaca','CodeMix-dataset']
keys = [{'question':'instruction', 'code':'output'}, {'question':'instruction', 'code':'output'}, {'question':'problem', 'code':'solution'}, {'question':'instruction', 'code':'response'},{'question':'instruction', 'code':'output'}, {'question':'question', 'code':'code'}]
for idx, i in enumerate(dataset_vars):
    ds_name = name_mapping[idx]
    q_key = keys[idx]['question']
    c_key = keys[idx]['code']
    print(f'Starting for {ds_name}')
    if ds_name=='magicoder_instruct':
        new_ds = [t for t in i if t['lang']=='python']
        question_mtld, code_mtld, question_spread, code_spread = calculate_diversity_score(new_ds,model,ds_name, question_key=q_key, code_key=c_key)
    else:
        question_mtld, code_mtld, question_spread, code_spread = calculate_diversity_score(i,model,ds_name,question_key=q_key, code_key=c_key)
    result_dict[ds_name] = {'question_mtld':question_mtld, 'code_mtld':code_mtld, 'question_spread':float(question_spread), 'code_spread':float(code_spread)}
    with open('diversity_results_v2.json', 'w') as json_file:
        json.dump(result_dict, json_file, indent=4)

Starting for evol_codealpaca_v1
Starting for new dataset


Calculating MTLD!
Length of probs = 0
Length of MTLD list = 57997

Length of probs = 0
Length of MTLD list = 57997
MTLD Done!
Preparing embeddings!


Batches:   0%|          | 0/10 [00:00<?, ?it/s]

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

Embedings done!
Calculating spread!
Started Cosine Sim
Finished Cosine Sim
Started Cosine Sim
Finished Cosine Sim
Spread calc done!
Starting for evol_instruct_code_v1
Starting for new dataset


Calculating MTLD!
Length of probs = 0
Length of MTLD list = 26588

Length of probs = 21
Length of MTLD list = 26567
MTLD Done!
Preparing embeddings!


Batches:   0%|          | 0/10 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

Embedings done!
Calculating spread!
Started Cosine Sim
Finished Cosine Sim
Started Cosine Sim
Finished Cosine Sim
Spread calc done!
Starting for CodeMix-dataset
Starting for new dataset


Calculating MTLD!
Length of probs = 1
Length of MTLD list = 20251

Length of probs = 0
Length of MTLD list = 20252
MTLD Done!
Preparing embeddings!


Batches:   0%|          | 0/10 [00:00<?, ?it/s]

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

Embedings done!
Calculating spread!
Started Cosine Sim
Finished Cosine Sim
Started Cosine Sim
Finished Cosine Sim
Spread calc done!


In [103]:
result_dict

{'evol_codealpaca_v1': {'question_mtld': 59.06801438647717,
  'code_mtld': 35.26156405023193,
  'question_spread': 0.7789011597633362,
  'code_spread': 0.8093565702438354},
 'evol_instruct_code_v1': {'question_mtld': 42.450001621497854,
  'code_mtld': 24.204403985269664,
  'question_spread': 0.7883182168006897,
  'code_spread': 0.801061749458313},
 'magicoder_instruct': {'question_mtld': 36.31170034281009,
  'code_mtld': 22.907301603978148,
  'question_spread': 0.8156304955482483,
  'code_spread': 0.8202199935913086},
 'magicoder_evol_instruct': {'question_mtld': 59.073604193942906,
  'code_mtld': 35.26902291192707,
  'question_spread': 0.778901994228363,
  'code_spread': 0.8092754483222961},
 'code_alpaca': {'question_mtld': 30.92201141117365,
  'code_mtld': 14.948013597450808,
  'question_spread': 0.8183609247207642,
  'code_spread': 0.7745630145072937},
 'CodeMix-dataset': {'question_mtld': 40.35980535761259,
  'code_mtld': 12.020248788664244,
  'question_spread': 0.7952884435653687

In [104]:
for i in result_dict:
    result_dict[i]['question_spread'] = 1/result_dict[i]['question_spread']
    result_dict[i]['code_spread'] = 1/result_dict[i]['code_spread']

In [109]:
{k:{p:round(q,2) for p,q in result_dict[k].items()} for k in result_dict}

{'evol_codealpaca_v1': {'question_mtld': 59.07,
  'code_mtld': 35.26,
  'question_spread': 1.28,
  'code_spread': 1.24},
 'evol_instruct_code_v1': {'question_mtld': 42.45,
  'code_mtld': 24.2,
  'question_spread': 1.27,
  'code_spread': 1.25},
 'magicoder_instruct': {'question_mtld': 36.31,
  'code_mtld': 22.91,
  'question_spread': 1.23,
  'code_spread': 1.22},
 'magicoder_evol_instruct': {'question_mtld': 59.07,
  'code_mtld': 35.27,
  'question_spread': 1.28,
  'code_spread': 1.24},
 'code_alpaca': {'question_mtld': 30.92,
  'code_mtld': 14.95,
  'question_spread': 1.22,
  'code_spread': 1.29},
 'CodeMix-dataset': {'question_mtld': 40.36,
  'code_mtld': 12.02,
  'question_spread': 1.26,
  'code_spread': 1.23}}

In [33]:
import pickle
min_range = 0.7775569693909751
max_range = 0.9527710969170251
with open('/workspace/CS762_Project/Notebooks/code_embeddings_magicoder_evol_instruct.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    stored_sentences = stored_data['sentences']
    stored_embeddings = stored_data['embeddings']



In [8]:
type(stored_embeddings)

numpy.ndarray

In [15]:
questions_spread = []
from tqdm import tqdm
for idx in tqdm(range(len(stored_embeddings))):
    embeddings1 = stored_embeddings[idx]
    embeddings2 = np.concatenate((stored_embeddings[:idx],stored_embeddings[idx+1:]), axis=0)
    question_spread = normalize_score(calculate_spread_cosine(embeddings1,embeddings2,'evol_codealpaca_v1', 'question' ), min_range, max_range)
    questions_spread.append(question_spread)
    print(question_spread)

  0%|          | 0/111272 [00:00<?, ?it/s]

Started Cosine Sim


  0%|          | 1/111272 [00:01<37:23:02,  1.21s/it]

torch.Size([1, 111271])
Done with Cosine Sim
0.009503579465811164
Started Cosine Sim


  0%|          | 2/111272 [00:02<40:39:57,  1.32s/it]

...

torch.Size([1, 111271])
Done with Cosine Sim
-0.09096433152936571
Started Cosine Sim


  0%|          | 64/111272 [01:20<37:22:56,  1.21s/it]

torch.Size([1, 111271])
Done with Cosine Sim
-0.13353467304525418
Started Cosine Sim


  0%|          | 64/111272 [01:21<39:26:33,  1.28s/it]


KeyboardInterrupt: 

In [1]:
import pickle
with open('question_similarity_evol_codealpaca_v1.pkl', "rb") as fIn:
    data = pickle.load(fIn)

EOFError: Ran out of input

In [34]:
import numpy as np

def average_std(embeddings):
    # Convert the list of embeddings to a NumPy array
    # Assuming each embedding is already a NumPy array
    embeddings_array = np.array(embeddings)

    # Calculate standard deviation along each axis/dimension
    std_devs = np.std(embeddings_array, axis=0)

    # Calculate the average of these standard deviations
    avg_std_dev = np.mean(std_devs)
    return avg_std_dev

# Example usage
embeddings = [np.random.rand(768) for _ in range(10)]  # 10 random embeddings of size 768
avg_std = average_std(stored_embeddings)
print("Average Standard Deviation:", avg_std)


Average Standard Deviation: 0.016798666


In [36]:
name_mapping = ['evol_codealpaca_v1','evol_instruct_code_v1','magicoder_instruct','magicoder_evol_instruct','code_alpaca','CodeMix-dataset']
std_results = {'question':{}, 'code':{}}
for i in name_mapping:
    for j in ['question', 'code']:
        with open(f'/workspace/CS762_Project/Notebooks/{j}_embeddings_{i}.pkl', "rb") as fIn:
            stored_data = pickle.load(fIn)
            stored_embeddings = stored_data['embeddings']
        avg_std = average_std(stored_embeddings)
        std_results[j][i] = avg_std

In [37]:
std_results

{'question': {'evol_codealpaca_v1': 0.01743797,
  'evol_instruct_code_v1': 0.017142734,
  'magicoder_instruct': 0.015445295,
  'magicoder_evol_instruct': 0.017438613,
  'code_alpaca': 0.016853206,
  'CodeMix-dataset': 0.016257199},
 'code': {'evol_codealpaca_v1': 0.01679833,
  'evol_instruct_code_v1': 0.017222662,
  'magicoder_instruct': 0.015257552,
  'magicoder_evol_instruct': 0.016798666,
  'code_alpaca': 0.017566217,
  'CodeMix-dataset': 0.015647808}}

In [56]:
import json
with open('diversity_results.json', 'r') as fp:
    diversity_results = json.load(fp)

In [57]:
diversity_results

{'evol_codealpaca_v1': {'question_mtld': 60.135132380757035,
  'code_mtld': 43.03108253698388},
 'evol_instruct_code_v1': {'question_mtld': 41.28180945403313,
  'code_mtld': 22.895224186569184},
 'magicoder_instruct': {'question_mtld': 36.31170034281009,
  'code_mtld': 22.907301603978148},
 'magicoder_evol_instruct': {'question_mtld': 60.14508636787472,
  'code_mtld': 43.044602836118024},
 'code_alpaca': {'question_mtld': 29.412054749714912,
  'code_mtld': 14.624050036777174},
 'CodeMix-dataset': {'question_mtld': 40.35980535761259,
  'code_mtld': 12.020248788664244}}